In [1]:
from semantic_router.encoders import HuggingFaceEncoder
from semantic_chunkers import StatisticalChunker

encoder = HuggingFaceEncoder(
    name = "NeuML/pubmedbert-base-embeddings"
)
chunker = StatisticalChunker(encoder=encoder)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Archit\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

def generate_chunks(file_path: str, chunker)-> list:
    
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()

    document_text = "\n".join([page.page_content for page in pages])

    chunks = chunker(docs=[document_text])
    chunked_data = []

    for chunk in chunks[0]:  
        combined_chunk = " ".join(chunk.splits)  
        chunked_data.append(Document(page_content=combined_chunk)) 

    return chunked_data

In [46]:
chunked_data = generate_chunks("../../dataset/pdfs/fm/1.pdf", chunker)

2024-10-18 18:16:53 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 300. Splitting to sentences before semantically merging.


  0%|          | 0/678 [00:00<?, ?it/s]

In [6]:
from langchain.graphs import Neo4jGraph
from dotenv import load_dotenv
import os 

load_dotenv()

graph = Neo4jGraph(
    url = 'neo4j+s://2e177e83.databases.neo4j.io',
    username = os.getenv('NEO4J_USERNAME'),
    password = os.getenv('NEO4J_PASSWORD'),
)

In [47]:
import json

with open('../../dataset/jsons/ent/ent_1.json', 'r') as file:

    data = json.load(file)

In [48]:
len(data)

191

In [49]:
len(chunked_data)

1801

In [43]:
from langchain.graphs.graph_document import GraphDocument, Node, Relationship
from langchain.schema import Document

graph_docs = []

for i, chunk in enumerate(data):

    node_list = []

    if len(chunk['nodes'])==0 and len(chunk['relationships'])==0:
        continue
    

    for node in chunk['nodes']:
        node = Node(id=node['id'], type=node['type'], properties = node['properties'])
        node_list.append(node)


    relationship_list  = []

    try:
        source_doc = Document(page_content = chunked_data[i].page_content, metadata = chunked_data[i].metadata)
    except Exception as e:
        source_doc = Document(page_content = "none", metadata= {})


    temp_doc = GraphDocument(
        nodes =  node_list,
        relationships = relationship_list,
        source = source_doc   
    )

    graph_docs.append(temp_doc)

In [50]:
len(graph_docs)

53

In [45]:
graph.add_graph_documents(
    graph_docs,
    baseEntityLabel=True,
    include_source=True
)